In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2

import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from PIL import Image

In [9]:
base_path = "../Data/CK_CK+/CK+/"

In [12]:
data_path = base_path + 'cohn-kanade-images'
emotion_path = base_path + 'Emotion'
facs_path = base_path + 'FACS'
landmarks_path = base_path + 'Landmarks'
neutralised_path = "NeutralisedImages/CK_CK+/CK+/" + 'cohn-kanade-images'

all_image_files = glob.glob(data_path+'/**/*.png', recursive=True)
all_emotion_files = glob.glob(emotion_path+'/**/*.txt', recursive=True)
all_facs_files = glob.glob(facs_path+'/**/*.txt', recursive=True)
all_landmarks_files = glob.glob(landmarks_path+'/**/*.txt', recursive=True)

In [13]:
all_emotions = {0: 'neutral', 1: 'anger', 2: 'contempt', 3: 'disgust', 
                4: 'fear', 5: 'happy', 6: 'sadness', 7: 'surprise'}

In [ ]:
image_dict = {}

for i_im in all_image_files:
    im_dict = {}
    im_folder = i_im.rsplit('/', 1)[0]
    if im_folder not in image_dict.keys():
        im_files = glob.glob(im_folder+'/*.png')
        im_dict['emotion'] = im_files[-1]
        im_dict['neutral'] = im_files[0]

        im_folder_base = "/".join(im_files[-1].rsplit('/',3)[-3:]).rsplit('/',1)[0]
        facs_folder = facs_path + '/' + im_folder_base
        emotion_folder = emotion_path + '/' + im_folder_base
        landmarks_folder = landmarks_path + '/' + im_folder_base
        
        facs_file = glob.glob(facs_folder+'/*.txt')[0]
        emotion_file = glob.glob(emotion_folder+'/*.txt')
        neutral_landmarks_file = glob.glob(landmarks_folder+'/*.txt')[0]
        emotion_landmarks_file = glob.glob(landmarks_folder+'/*.txt')[-1]

        with open(facs_file) as f:
            data = f.read()
            im_dict['facs'] = np.array([list(map(float, i.split())) for i in data.strip().split('\n')])

        if len(emotion_file) > 0:
            emotion_file = emotion_file[0]
            with open(emotion_file) as f:
                data = f.read()
                im_dict['label'] = int(float(data))
                im_dict['label_str'] = all_emotions[im_dict['label']]

        with open(neutral_landmarks_file) as f:
            data = f.read()
            im_dict['neutral_landmarks'] = np.array([list(map(float, 
                                                     i.split())) for i in data.strip().split('\n')])

        with open(emotion_landmarks_file) as f:
            data = f.read()
            im_dict['emotion_landmarks'] = np.array([list(map(float, 
                                                     i.split())) for i in data.strip().split('\n')])

    #     print(im_dict)
        image_dict[im_folder] = (im_dict)

In [ ]:
df_train = df_images[~df_images['label'].isna()][['im_neutral_crop', 'im_emotion_crop', 
                                                  'neutral_crop_landmarks', 'emotion_crop_landmarks']]
label_string = df_train.index
label_string = label_string.str.split('/')
df_train['label'] = [int(i[-2][1:]) for i in label_string]
df_train.reset_index(drop=True, inplace=True)
df_train['im_emotionless'] = df_train.apply(lambda row: apply_TPS(row, tps), axis=1)

In [ ]:
X, y = df_train[['im_neutral_crop', 'im_emotion_crop', 'im_emotionless']], df_train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.reset_index(drop=True, inplace=True), X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True), y_test.reset_index(drop=True, inplace=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Fisher Face

In [ ]:
fishface = cv2.face.FisherFaceRecognizer_create() #Initialize fisher face classifier
fishface.train(X_train['im_neutral_crop'], np.array(y_train))
correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotion_crop']):
    pred, conf = fishface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotion Acc: ', (100*correct)/(correct + incorrect))

correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotionless']):
    pred, conf = fishface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotionless Acc: ', (100*correct)/(correct + incorrect))

In [ ]:
print(fishface.predict(X_test['im_emotion_crop'][10]), y_test[10], 
      fishface.predict(X_test['im_emotionless'][10]), y_test[10])

# Eigen Face

In [ ]:
eigenface = cv2.face.EigenFaceRecognizer_create()
eigenface.train(X_train['im_neutral_crop'], np.array(y_train))
correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotion_crop']):
    pred, conf = eigenface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotion Acc: ', (100*correct)/(correct + incorrect))

correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotionless']):
    pred, conf = eigenface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotionless Acc: ', (100*correct)/(correct + incorrect))

In [ ]:
print(eigenface.predict(X_test['im_emotion_crop'][10]), y_test[10], 
      eigenface.predict(X_test['im_emotionless'][10]), y_test[10])

# LBHP 

In [ ]:
lbph_face = cv2.face.LBPHFaceRecognizer_create()
lbph_face.train(X_train['im_neutral_crop'], np.array(y_train))
correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotion_crop']):
    pred, conf = lbph_face.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotion Acc: ', (100*correct)/(correct + incorrect))

correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotionless']):
    pred, conf = lbph_face.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotionless Acc: ', (100*correct)/(correct + incorrect))

In [ ]:
print(lbph_face.predict(X_test['im_emotion_crop'][10]), y_test[10], 
      lbph_face.predict(X_test['im_emotionless'][10]), y_test[10])